In [16]:
# Loading Data and Packages
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import LogLikelihoodTorch
import torch
from plotly.subplots import make_subplots
import plotly.graph_objects as go


with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f)
torch.set_default_dtype(torch.float64)
X_tensor = torch.from_numpy(X).to(torch.float64)

labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values

with open('EM_K=7.pickle','rb') as f:
    Pis_EM,kappas_EM,mus_EM,states_EM = pickle.load(f)

with open('Torch_K=7.pickle','rb') as f:
    Pis_T,kappas_T,mus_T = pickle.load(f)

# Calculate states of Torch Optimized Model
B = torch.exp(torch.log(Pis_T)+LogLikelihoodTorch.log_pdf(X_tensor,mus_T,kappas_T,90)).T
B=B/B.sum(0)
states_T=B.max(0).indices
ComparisonIndex = [(0,5),(1,1),(2,4),(3,6),(4,0),(5,3),(6,2)]
for idx,state in enumerate(states_T):
    states_T[idx] = ComparisonIndex[state.item()][1]

In [17]:
# Define Time for plots
Time = np.arange(330)*3/4

In [18]:
# Create a plot with subplots comparing two first individuals for each model
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(
            x=Time,
            y=states_T[0:330],
            mode='lines+markers',
            name="Subject 1 Torch"
            ),
    
    row=1,col=1)

fig.add_trace(go.Scatter(
                x=Time,
                y=states_EM[0:330],
                mode='lines+markers',
                name="Subject 1 EM"),
                1,1)

fig.add_trace(go.Scatter(
            x=Time,
            y=states_T[330:660],
            mode='lines+markers',
            name="Subject 2 Torch"),
            2,1)

fig.add_trace(go.Scatter(
            x=Time,
            y=states_EM[330:660],
            mode='lines+markers',
            name="Subject 2 EM"),
            2,1)


fig.update_layout(height=800, width=1200,
                title_text=None,
                showlegend=True)
fig.show()

In [9]:
# Comparing Likelihood

log_Likelihood_EM =  LogLikelihoodTorch.log_likelihood_eval(X_tensor,torch.from_numpy(Pis_EM),torch.from_numpy(kappas_EM),torch.from_numpy(mus_EM),p=90,K=7)
log_Likelihood_torch = LogLikelihoodTorch.log_likelihood_eval(X_tensor,Pis_T,kappas_T,mus_T,p=90,K=7)

print("Likelihood Camparison:")
print(f"EM-Algortihem LogLikelihood: L = {log_Likelihood_EM}")
print(f"Torch Optimization of LogLikelihood: L = {log_Likelihood_torch}")

Pis_T,kappas_T,mus_T = Pis_T.detach().numpy(),kappas_T.detach().numpy(),mus_T.detach().numpy()

Likelihood Camparison:
EM-Algortihem LogLikelihood: L = 6469526.793269191
Torch Optimization of LogLikelihood: L = 7204118.597278038


In [76]:
# Analytical paring of Centroids
ComparisonIndex = [(0,5),(1,1),(2,4),(3,6),(4,0),(5,3),(6,2)]

# Plot each paired centroid
for Index in ComparisonIndex:
    fig = make_subplots(rows=1, cols=2,
                        shared_yaxes=True,
                        subplot_titles=(f" pi = {np.round(Pis_T[Index[0]],3)}, kappa = {np.round(kappas_T[Index[0]],1)} ",f" pi = {np.round(Pis_EM[Index[0]],3)}, kappa = {np.round(kappas_EM[Index[0]],1)}"))

    fig.add_trace(go.Bar(
                y=label,
                x=mus_T[:,Index[0]],
                orientation='h'),
                1,1)

    fig.add_trace(go.Bar(
                y=label,
                x=mus_EM[:,Index[1]],
                orientation='h'),
                1,2)


    fig.update_layout(height=1500, width=800,
                    title_text=".             Torch Log-Likelihood Optimization      Expectation-maximization Algortihem",
                    showlegend=False)
    fig.show()